In [15]:
import pandas as pd
df = pd.read_csv('preprocessed_lyrics.csv')
df = df.groupby('Genre', group_keys=False).apply(lambda s: s.sample(5000, random_state=42)) #


In [16]:
df

,SName,Lyric,Genre,Artist,lyric_length_processed
2157,Family Affair,refrain let crunkupon gon fun dancery ya open ...,Hip Hop,Mary J. Blige,244
3187,Ransom (ft. Lil' Wayne),ransom yeah Drizzy Baby know time high time sm...,Hip Hop,Drake,426
9164,Close To Me,Cent unstoppable incredible impeccable unit wh...,Hip Hop,G-Unit,242
2145,Zone,uhh yea uh uh uh alright alright love bump bad...,Hip Hop,Drake,164
6442,"Why You Up In Here (feat. Ludacris, Git Fresh ...",Flo Rida Gucci bird buy ciroc lil mama jock st...,Hip Hop,Flo Rida,199
...,...,...,...,...,...
31738,What do You Need?,need tonight feel look right pretend right may...,Rock,Goo Goo Dolls,67
31425,Rebel Heart,Stewart Golub Kentis Rojas pick quarter brothe...,Rock,Rod Stewart,181
35407,Before The Dawn,meet dark hold maybe tonight fly far away lose...,Rock,Evanescence,43
35281,Spanish is the Loving Tongue,break heart lose soul Adios mi cora sole spani...,Rock,Bob Dylan,65


In [17]:
df = df[df['Lyric'].notna()]


In [18]:
from sklearn.model_selection import train_test_split
 
train, test = train_test_split(df, test_size=0.2, random_state=42)
X_train = train.loc[:, train.columns != 'Genre']
y_train = train['Genre']
X_test = test.loc[:, test.columns != 'Genre']
y_test = test['Genre']

In [19]:
X_train

,SName,Lyric,Artist,lyric_length_processed
17421,Party At A Rich Dude’s House,swimming pool sink come let come let cause sce...,Kesha,154
16209,Telephone/Bad Romance,hello hello baby hear thing service club oh br...,Alphabeat,131
22987,Revenge For Love,need saviour bail hold breath water let alimon...,Spandau Ballet,59
28732,Never Be The Same (Feat. Kane Brown) (Remix),wrong brain chemical vein feel high feel pain ...,Camila Cabello,125
3608,On My Own,Les Nubians go build bridge original creation ...,Black Eyed Peas,126
...,...,...,...,...
26875,Underneath Your Clothes,song write hand God wrong cause sound bit odd ...,Shakira,86
35405,The Garden,know friend home everybody garden look tree lo...,Guns N' Roses,109
25498,Baby,ohh wooaah know love know care shout want love...,Austin Mahone,188
4043,Rules,hoe cryin bitch niggaz cryin hoe scratch sampl...,Wu-Tang Clan,356


In [20]:
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin

class PairedSentenceVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    # Vectorize a single sentence.
    def _transform1(self, sentence):
        sentence = sentence.split()
        vector = np.array((sum(nlp.vocab[word].vector for word in sentence))/len(sentence))
        return (vector)

    # Vectorize a single row of the dataframe.
    def _transform2(self, row):
        # TODO: Replace the following line with your own code
        sentence = self._transform1(row.Lyric)
        return sentence

    def transform(self, X):
        return np.concatenate(
            [self._transform2(row).reshape(1, -1) for row in X.itertuples()]
        )

In [ ]:
# TODO: Enter code here to implement the word embeddings classifier. Print the classification report.
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import spacy
#!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')
from sklearn.neural_network import MLPClassifier
pipe = Pipeline(steps=[('vectorizer', PairedSentenceVectorizer()),
                       ('clf', MLPClassifier(random_state=1,  max_iter=8, hidden_layer_sizes=(300, 300, 300)))])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
target_names = df['Genre'].unique()

print(classification_report(y_test, y_pred, target_names=target_names))